## Step 1: Importing  Data 

The Data is in json format so i used Json package to read the json file 

In [1]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\saidh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\saidh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Using TensorFlow backend.
C:\Users\saidh\miniconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\saidh\miniconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\saidh\miniconda3\envs\tensor

## Step2:Preprocessing Data

Iterated the patterns using nltk.word_tokenize() function and append each word in the list 
Also created list of classes for our tags

In [2]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)#tokenize each word
        words.extend(w)
        documents.append((w, intent['tag']))#add documents in the corpus       
        if intent['tag'] not in classes:# add to our classes list
            classes.append(intent['tag'])

## Step 3:Now lemmatize each word and remove duplicate words from the list 
Lemmatizing: it is the process of converting word into its lemma form 
I created a pickle file to store the trained data so that we no need to train the model each and every time 
we want to run the code

In [3]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]# lemmaztize and lower each word and remove duplicates
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))# sort classes
print (len(documents), "documents")# documents = combination between patterns and intents
print (len(classes), "classes", classes)# classes = intents
print (len(words), "unique lemmatized words", words)# words = all words, vocabulary

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
88 unique lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


## Step 4:Create training and testing data

our input will be the pattern and o/p will be the classs of our i/p
Compuer understands only numbers so we convert words into numbers 

In [4]:
training = [] # create our training data
output_empty = [0] * len(classes) # create an empty array for our output
for doc in documents:# training set, bag of words for each sentence
    bag = [] # initialize our bag of words
    pattern_words = doc[0] # list of tokenized words for the pattern
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]    # lemmatize each word 
    for w in words:    # create our bag of words array with 1, if word match found in current pattern
        bag.append(1) if w in pattern_words else bag.append(0)
    
    output_row = list(output_empty)   # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
    
random.shuffle(training)# shuffle our features and turn into np.array
training = np.array(training)

train_x = list(training[:,0])# create train lists. X - patterns, Y - intents
train_y = list(training[:,1])# create test lists. X - patterns, Y - intents
print("Training data created")

Training data created


C:\Users\saidh\miniconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  app.launch_new_instance()


## Step5:
now i bulded the model with neural network that has 3 layers                                                                     
First layer 128 neurons, second layer 64 neurons and 3rd output layer                                                           
i have given 200 epochs so neural network trains 200 times 
as i gave less data our accuracy is 100%                      

In [5]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])#Stochastic gradient descent gives good result

#Save the model so we no need to train the model every time 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")


Epoch 1/200
47/47 [==============================] - 1s 25ms/step - loss: 2.1595 - acc: 0.1064
Epoch 2/200
47/47 [==============================] - 0s 616us/step - loss: 2.1688 - acc: 0.1277
Epoch 3/200
47/47 [==============================] - 0s 552us/step - loss: 2.0579 - acc: 0.2766
Epoch 4/200
47/47 [==============================] - 0s 615us/step - loss: 1.9669 - acc: 0.4043
Epoch 5/200
47/47 [==============================] - 0s 594us/step - loss: 1.8997 - acc: 0.4255
Epoch 6/200
47/47 [==============================] - 0s 531us/step - loss: 1.6978 - acc: 0.6170
Epoch 7/200
47/47 [==============================] - 0s 552us/step - loss: 1.6042 - acc: 0.5319
Epoch 8/200
47/47 [==============================] - 0s 509us/step - loss: 1.5182 - acc: 0.5745
Epoch 9/200
47/47 [==============================] - 0s 573us/step - loss: 1.2514 - acc: 0.6809
Epoch 10/200
47/47 [==============================] - 0s 594us/step - loss: 1.2779 - acc: 0.6170
Epoch 11/200
47/47 [====================

## Step6:We will load the trained model

The model will only tell us the class it belongs to, so implement some functions which will identify the class 
and then retrieve us a random response from the list of responses.


In [6]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np
import tensorflow as tf
from keras.models import load_model

model =tf.keras.models.load_model('chatbot_model.h5', compile=False)
import json
import random
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

ModuleNotFoundError: No module named 'tf'

## Step7:

Create some functions that will perform text preprocessing and then predict the class.

In [7]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)# tokenize the pattern - split words into array
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]# stem each word
    return sentence_words

## 0 or 1 for each word in the bag that exists in the sentence

In [8]:
def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)# tokenize the pattern
    bag = [0]*len(words) # bag of words - matrix of N words, vocabulary matrix 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1 # assign 1 if current word is in the vocabulary position
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

In [9]:
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

## Step8:
After predicting the class,Get a random response from the list of intents.

In [10]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

In [11]:
def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

## GUI

In [12]:
import tkinter
from tkinter import *


def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
    
        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')
            
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
 

base = Tk()
base.title("Hello")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()

In [9]:
from tensorflow import keras
from keras.models import load_model